In [3]:
!pip install deap

   ---------------------------------------- 0.0/109.3 kB ? eta -:--:--
   ---------------------------------------- 109.3/109.3 kB 6.6 MB/s eta 0:00:00


In [4]:
import random
from deap import base, creator, tools, algorithms
import numpy as np
import pandas as pd


In [5]:
import pandas as pd
import numpy as np

# Define the number of samples
num_samples = 1000

# Define ranges for each variable
ranges = {
    'Inlet Air Temperature (°C)': (150, 250),
    'Inlet Air Humidity (%)': (30, 80),
    'Feed Flow Rate (mL/min)': (50, 200),
    'Atomization Pressure (bar)': (5, 20),
    'Coconut Milk Solids Content (%)': (10, 20),
    'Moisture Content of Dried Powder (%)': (1, 5),
    'Particle Size Distribution (μm)': (10, 100)
}

# Generate random data within defined ranges
data = {}
for var, (min_val, max_val) in ranges.items():
    data[var] = np.random.uniform(min_val, max_val, num_samples)

# Create a DataFrame from the generated data
df = pd.DataFrame(data)

# Display the first few rows of the dataset
print(df.head())

# Save the dataset to a CSV file
df.to_csv('sample_dataset.csv', index=False)


   Inlet Air Temperature (°C)  Inlet Air Humidity (%)  \
0                  168.333514               64.092543   
1                  172.683103               53.217140   
2                  165.892649               67.421686   
3                  159.300431               45.922626   
4                  175.640701               60.218926   

   Feed Flow Rate (mL/min)  Atomization Pressure (bar)  \
0                73.014639                   10.970004   
1               156.521241                   12.810047   
2               160.762925                   15.234965   
3               163.476402                   13.945703   
4               114.957519                   14.076866   

   Coconut Milk Solids Content (%)  Moisture Content of Dried Powder (%)  \
0                        15.525258                              2.830684   
1                        13.124299                              1.917562   
2                        12.172767                              1.889116   
3   

In [6]:
# Load the dataset
df = pd.read_csv("sample_dataset.csv")


In [7]:
# Define evaluation function for the hybrid model
def evaluate(individual):
    # Extract parameters from the individual (e.g., neural network architecture parameters)
    num_neurons, num_layers = individual

    # Split the dataset into features (X) and target (y)
    X = df.drop(columns=["Moisture Content of Dried Powder (%)", "Particle Size Distribution (μm)"])
    y = df["Moisture Content of Dried Powder (%)"]

    # Train-test split
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define and compile the neural network model
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense

    model = Sequential()
    model.add(Dense(num_neurons, input_dim=X_train.shape[1], activation='relu'))
    for _ in range(num_layers - 1):
        model.add(Dense(num_neurons, activation='relu'))
    model.add(Dense(1))  # Output layer with a single neuron for regression
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model
    mse = model.evaluate(X_test, y_test, verbose=0)
    fitness = 1 / (1 + mse)  # Higher fitness for lower MSE (better performance)

    return fitness,

In [8]:
# Define genetic algorithm parameters
POPULATION_SIZE = 10
GENERATIONS = 1

In [9]:
# Create types for fitness and individuals in the genetic algorithm
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize toolbox
toolbox = base.Toolbox()

# Define attribute generators for the individual (neural network architecture parameters)
toolbox.register("attr_neurons", random.randint, 1, 100)
toolbox.register("attr_layers", random.randint, 1, 5)

# Define individual and population creation methods
toolbox.register("individual", tools.initCycle, creator.Individual, (toolbox.attr_neurons, toolbox.attr_layers), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Genetic operators
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=100, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Create initial population
population = toolbox.population(n=POPULATION_SIZE)

# Run the genetic algorithm
for gen in range(GENERATIONS):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)

    fitnesses = toolbox.map(toolbox.evaluate, offspring)
    for ind, fit in zip(offspring, fitnesses):
        ind.fitness.values = fit

    population = toolbox.select(offspring, k=len(population))

c:\Users\sanja\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\sanja\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\sanja\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in

In [10]:
# Extract the best individual (best model parameters)
best_individual = tools.selBest(population, k=1)[0]
best_params = best_individual

print("Best Parameters:", best_params)


Best Parameters: [50, 5]
